In [5]:
import cv2
import numpy as np

<div style="display: flex;">
    <figure>
        <img src="assets/img1.jpg" width="400"/>
        <figcaption style="text-align: center;">图片1</figcaption>
    </figure>
    <figure>
        <img src="assets/img2.jpg" width="400"/>
        <figcaption style="text-align: center;">图片2</figcaption>
    </figure>
</div>
<div style="display: flex;">
    <figure>
        <img src="assets/img3.jpg" width="400"/>
        <figcaption style="text-align: center;">图片3</figcaption>
    </figure>
    <figure>
        <img src="assets/img4.jpg" width="400"/>
        <figcaption style="text-align: center;">图片4</figcaption>
    </figure>
</div>
<div style="display: flex;">
    <figure>
        <img src="assets/img5.jpg" width="400"/>
        <figcaption style="text-align: center;">图片5</figcaption>
    </figure>
    <figure>
        <img src="assets/img6.jpg" width="400"/>
        <figcaption style="text-align: center;">图片6</figcaption>
    </figure>
</div>

In [6]:
class Stitcher:

    # 拼接函数
    def stitch(self, images, ratio = 0.75, reprojThresh = 4.0, showMatches = True):
        # 读取图像
        imageA, imageB = images
        # 计算特征点和特征向量
        kpsA, featureA = self.detectAndDescribe(imageA)
        kpsB, featureB = self.detectAndDescribe(imageB)

        # 匹配两张图片的特征点
        M = self.matchKeypoints(kpsA, kpsB, featureA, featureB, ratio, reprojThresh)

        # 没有匹配点，退出
        if not M:
            return None

        matches, H, status = M
        # 将图片B进行视角变换 中间结果
        result = cv2.warpPerspective\
            (imageB, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
        cv2.imshow('h',result)
        # 将图片A传入
        result[0:imageA.shape[0], 0:imageA.shape[1]] = imageA

        # 计算混合渐变
        mask = np.zeros((imageA.shape[0], imageA.shape[1]), dtype=np.uint8)
        mask[:, imageA.shape[1]-40:] = 255
        result = cv2.seamlessClone(imageA, result, mask, (imageA.shape[1], imageA.shape[0]//2), cv2.NORMAL_CLONE)
        
        result = self.trim_border(result)

        # 检测是否需要显示图片匹配
        if showMatches:
            # 生成匹配图片
            vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
            # 返回结果A
            return result, vis

        # 返回匹配结果
        return result


    def detectAndDescribe(self, image):
        # 转换为灰度图
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 建立SIFT生成器
        descriptor = cv2.SIFT_create()
        # 检测特征点并计算描述子
        kps, features = descriptor.detectAndCompute(gray, None)

        kps = np.float32([kp.pt for kp in kps])

        return kps, features

    def matchKeypoints(self, kpsA, kpsB, featureA, featureB, ratio, reprojThresh):
        # 建立暴力匹配器
        matcher = cv2.BFMatcher()

        # 使用KNN检测来自AB图的SIFT特征匹配
        rawMatches = matcher.knnMatch(featureA, featureB, 2)

        # 过滤
        matches = []
        for m in rawMatches:
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))

        if len(matches) > 4:
            # 获取匹配对的点坐标
            ptsA = np.float32([kpsA[i] for (_, i) in matches])
            ptsB = np.float32([kpsB[i] for (i, _) in matches])

            # 计算H矩阵
            H, status = cv2.findHomography(ptsB, ptsA, cv2.RANSAC, reprojThresh)

            return matches, H, status

    # 展示图像
    def cv_show(self,name, img):
        cv2.imshow(name, img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
        # 初始化可视化图片，将A、B图左右连接到一起
        hA, wA = imageA.shape[:2]
        hB, wB = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:(wA+wB)] = imageB

        # 联合遍历，画出匹配对
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # 当点对匹配成功时，画到可视化图上
            if s == 1:
                # 画出匹配对
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        # 返回可视化结果
        return vis
    
    def trim_border(self, img):
        # 转换为灰度图像
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 二值化
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # 寻找轮廓
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 找到最大的轮廓
        max_area = 0
        max_contour = None
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > max_area:
                max_area = area
                max_contour = contour

        # 计算最大轮廓的边界矩形
        x, y, w, h = cv2.boundingRect(max_contour)

        # 使用透视变换裁剪图像
        src_points = np.float32([[x, y], [x + w, y], [x, y + h], [x + w, y + h]])
        dst_points = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        cropped_img = cv2.warpPerspective(img, M, (w, h))

        return cropped_img

In [7]:
# 读取图片
img1 = cv2.imread('./assets/img2.jpg')
img2 = cv2.imread('./assets/img1.jpg')


# 图片拼接
stitcher = Stitcher()
result, vis = stitcher.stitch([img1, img2], showMatches=True)
# cv2.imshow('img1', img1)
# cv2.imshow('img2', img2)
cv2.namedWindow('keypoints matches',0)
cv2.resizeWindow('keypoints matches',(img1.shape[1]+img2.shape[1],img1.shape[0]))
cv2.imshow('keypoints matches',vis)
cv2.namedWindow('result',0)
cv2.resizeWindow('result',(img1.shape[1]+img2.shape[1],img1.shape[0]))
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.imwrite("./assets/result.jpg", result)

True

<div style="display: flex;">
    <figure>
        <img src="assets/result1.jpg" width="800"/>
        <figcaption style="text-align: center;">图片1和2拼接结果</figcaption>
    </figure>
</div>

In [8]:
# 读取图片
img1 = cv2.imread('./assets/img3.jpg')
img2 = cv2.imread('./assets/img4.jpg')


# 图片拼接
stitcher = Stitcher()
result, vis = stitcher.stitch([img1, img2], showMatches=True)
cv2.namedWindow('keypoints matches',0)
cv2.resizeWindow('keypoints matches',(img1.shape[1]+img2.shape[1],img1.shape[0]))
cv2.imshow('keypoints matches',vis)
cv2.namedWindow('result',0)
cv2.resizeWindow('result',(img1.shape[1]+img2.shape[1],img1.shape[0]))
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.imwrite("./assets/result2.jpg", result)

True

<div style="display: flex;">
    <figure>
        <img src="assets/result2.jpg" width="800"/>
        <figcaption style="text-align: center;">图片3和4拼接结果</figcaption>
    </figure>
</div>

In [10]:
# 读取图片
img1 = cv2.imread('./assets/img6.jpg')
img2 = cv2.imread('./assets/img6.jpg')

# 图片拼接
stitcher = Stitcher()
result, vis = stitcher.stitch([img1, img2], showMatches=True)
cv2.namedWindow('keypoints matches',0)
cv2.resizeWindow('keypoints matches',(img1.shape[1]+img2.shape[1],img1.shape[0]))
cv2.imshow('keypoints matches',vis)
cv2.namedWindow('result',0)
cv2.resizeWindow('result',(img1.shape[1]+img2.shape[1],img1.shape[0]))
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.imwrite("./assets/result3.jpg", result)

True

<div style="display: flex;">
    <figure>
        <img src="assets/result3.jpg" width="800"/>
        <figcaption style="text-align: center;">图片5和6拼接结果</figcaption>
    </figure>
</div>